In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls

data  task3_1_multilabel.ipynb


In [ ]:
#Change to the correct path if executed
%cd /content/gdrive/Shareddrives/NLI/Submission/Training

[Errno 2] No such file or directory: 'gdrive/Shareddrives/NLI/Alexander colabs'
/content/gdrive/Shareddrives/NLI/Alexander colabs/[T3]AM_DA


In [ ]:
%cd AM_DA

[Errno 2] No such file or directory: 'AM_DA'
/content/gdrive/Shareddrives/NLI/Alexander colabs/[T3]AM_DA


In [ ]:
!pip install datasets

In [ ]:
! pip install evaluate

In [ ]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yq05agpz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yq05agpz
  Resolved https://github.com/huggingface/transformers to commit 35478182ce50d04bde5c4ecd0569c2f6ba15bee7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! pip install accelerate -U

In [ ]:
!ls

data  task3_1_multilabel.ipynb


In [ ]:
# --test_file shall be provided only along with --do_predict
! python ../run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/agent_da_train_multilabel_parsed.json" --validation_file "data/agent_da_validation_multilabel_parsed.json" --test_file "data/agent_da_test_multilabel_parsed.json" --shuffle_train_dataset --metric_name accuracy --text_column_name "input" --label_column_name "output" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 1 --output_dir T3_DA

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
2023-12-11 19:04:10.806101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 19:04:10.806195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 19:04:10.806252: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 19:04:13.119399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/11/2023 19:04:17 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0, distributed training: False, 1

In [ ]:
##da
import ast
from sklearn.metrics import classification_report
import pandas as pd

df_test = pd.read_json("./T3_3_to_be_req_data/agent_tbreq_train_multilabel_parsed.json", lines = True)
df_test_preds = pd.read_csv("T3_DA/predict_results.txt", sep="\t")
true_labels = df_test["output"].tolist()
pred_labels = df_test_preds["prediction"].tolist()
output_list_of_lists = [ast.literal_eval(string) for string in pred_labels]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(true_labels)
y_gt_t = mlb.transform(true_labels)
y_pred_t =mlb.transform(output_list_of_lists)
print(classification_report(y_gt_t, y_pred_t, target_names=mlb.classes_))


#print(classification_report(true_labels, pred_labels))

                       precision    recall  f1-score   support

      booking-bookday       1.00      1.00      1.00        81
   booking-bookpeople       1.00      1.00      1.00        70
     booking-bookstay       1.00      1.00      1.00        62
     booking-booktime       1.00      1.00      1.00        58
           hotel-area       1.00      1.00      1.00      1575
        hotel-bookday       1.00      1.00      1.00       716
     hotel-bookpeople       1.00      1.00      1.00       400
       hotel-bookstay       1.00      1.00      1.00       592
       hotel-booktime       1.00      1.00      1.00        25
       hotel-internet       1.00      1.00      1.00       159
           hotel-name       1.00      1.00      1.00        68
        hotel-parking       1.00      1.00      1.00       207
     hotel-pricerange       1.00      1.00      1.00      1007
          hotel-stars       1.00      1.00      1.00       306
           hotel-type       1.00      1.00      1.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
text = "I am also looking for The Riverside Brasserie. | Restaurant-Inform | riverside brasserie is a modern european restaurant in the centre. Its address is Doubletree by Hilton Cambridge Granta Place Mill Lane and postcode is cb21rt. Want to book it? | Booking-Inform, Restaurant-Inform | Yes, I would like to book it and get a reference number."
text = "Sure, when would you like that reservation? | Booking-Request | i want to book it for 2 people and 2 nights starting from saturday. | Hotel-Inform"
gt_output = 'Restaurant-Request'

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = AutoModelForSequenceClassification.from_pretrained("T3_1_multilabeled_dataset_approach").to(device)
model_config = json.load(open("T3_1_multilabeled_dataset_approach/config.json", "r"))

encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

print('device:', device)
input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

print(predicted_labels)
pred_id = str(predicted_labels.cpu().detach().numpy()[0])
print(model_config["id2label"][pred_id])

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

device: cuda
tensor([5], device='cuda:0')
Booking-Book


In [ ]:
# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)
# Define a threshold
threshold = 0.3
# Apply threshold to determine label assignments
predicted_labels = (probabilities > threshold).int()
non_zero_positions = [i for i, value in enumerate(predicted_labels[0]) if value != 0]
for i in non_zero_positions:
  print(model_config["id2label"][str(i)])

Booking-Book
general-reqmore


In [ ]:
pred_id = str(predicted_labels.cpu().detach().numpy()[0])

In [ ]:
predicted_labels[0]

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0], device='cuda:0', dtype=torch.int32)

In [ ]:
non_zero_positions

[5, 23]

In [ ]:
predicted_labels[0]

tensor(5, device='cuda:0')

In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.5475, -4.6453, -4.8793, -4.2881, -4.7191, -4.4564, -2.2681, -4.5760,
         -4.3928, -5.0343, -4.7879, -4.3676, -4.3707, -4.5066, -4.3494, -4.8081,
          0.3569, -1.4332, -2.4580,  0.2261, -2.2659, -3.6159, -3.5394, -2.4451,
         -4.8814, -3.8986]], device='cuda:0'), hidden_states=None, attentions=None)

In [ ]:
logits

tensor([-4.5475, -4.6453, -4.8793, -4.2881, -4.7191, -4.4564, -2.2681, -4.5760,
        -4.3928, -5.0343, -4.7879, -4.3676, -4.3707, -4.5066, -4.3494, -4.8081,
         0.3569, -1.4332, -2.4580,  0.2261, -2.2659, -3.6159, -3.5394, -2.4451,
        -4.8814, -3.8986], device='cuda:0')

In [ ]:
aa = logits[0]

In [ ]:
aa[:25]

tensor([-4.5475, -4.6453, -4.8793, -4.2881, -4.7191, -4.4564, -2.2681, -4.5760,
        -4.3928, -5.0343, -4.7879, -4.3676, -4.3707, -4.5066, -4.3494, -4.8081,
         0.3569, -1.4332, -2.4580,  0.2261, -2.2659, -3.6159, -3.5394, -2.4451,
        -4.8814], device='cuda:0')

In [ ]:
CAUSE_INDICES = range(0, 26)

In [ ]:
len(CAUSE_INDICES)

26

In [ ]:
predicted_labels

tensor([16], device='cuda:0')

In [ ]:
print(predicted_labels)
pred_id = str(predicted_labels.cpu().detach().numpy()[0])
print(model_config["id2label"][pred_id])

tensor([16], device='cuda:0')
Restaurant-Inform
